# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: GroveStreet

Roll number: 2021701023

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import cv2
import copy
import open3d as o3d
from glob import glob

np.set_printoptions(suppress=True)

# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


<font color='blue' size=4.0/> 1) What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?

* **Affine Transformations** are of the form $f(x) = Ax + b$, where $A \in \mathbb R^{m \times n}$, $b \in \mathbb R^{m}$ for $x \in \mathbb R^{n}$. $A$ is a linear transformation matrix and $b$ represents the translation vector. Affine transformation preserve the collinearity of points, parallel lines, convexity of sets and ratio of length of parallel line segments. Scaling, Rotation, Translation, Reflection and Shear can all be derived from Affine transformation.
* **Similarity Transformations** have two forms, one for points and another one for frames. 
    * For points, we have $f(x) = rAx + t$, where $A$ is $n \times n$ orthogonal matrix, $r$ is a scalar, $t \in \mathbb R^{n}$ is translation vector and $x \in \mathbb R^{n}$. This is a special case of Affine transformation as matrix $A$ is constrained to be orthogonal to represent rotations. Here, points undergo translation, rotation and scaling. Shapes are preserved under similarity transformation for points.
    * For frames, we have $B = RAR^{-1}$, with $A,B,R \in \mathbb R^{n \times n}$. For our purpose, $n=3$, $A$ is linear transformation in Frame-$0$, $R$ is coordinate transform of Frame-$1$ w.r.t. Frame-$0$ and $B$ is the same linear transformation as $A$ in Frame-$1$. Under this definition, the eigenvalues of $A$ and $B$ are preserved.
* **Eucledian Transformation** has the form $f(x) = Rx + t$, here $R \in \mathbb R^{3 \times 3}$ is orthogonal matrix and $t \in \mathbb R^3$ is translation vector. They are special cases of affine transformation and similarity transformation (with $r=1$). This transformation preserves the shape, size and distance between points.

<font color='blue' size=4.0/> 2) Watch this video to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity?

Points at infinity are the points which are very far from the camera. They can be transformerd to points not at infinity by setting the last coordinate in homogenous coordinates to 0.

<font color='blue' size=4.0/> 3) Estimate the camera to world frame transformation matrix 

Transformation about current frame is represented by post-multiplication, whereas transformation about fixed frame is represented by pre-multiplication. Following are the transformation matrices after each step.
- A:  $T_{i}T_1$
- B:  $T_{2}T_{i}T_{1}$
- C:  $T_{3}T_{2}T_{i}T_{1}$
- D:  $T_{3}T_{2}T_{i}T_{1}T_{4}$
- E:  $T_{3}T_{2}T_{i}T_{1}T_{4}T_{5}$

Final transformation matrix is $T_{3}T_{2}T_{i}T_{1}T_{4}T_{5}$

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [2]:
odometry = np.loadtxt("./data/odometry.txt", delimiter=' ')
odometry = odometry[:77].reshape(77, 3, 4)

bin_files = glob("./data/LiDAR/*")
bin_files.sort()

total_frames = len(bin_files)

In [3]:
indices = np.random.choice(list(range(total_frames)), 2)
print(indices)

for index in indices:
    pcd = np.fromfile(bin_files[index], dtype=np.float32)
    pcd = pcd.reshape(-1, 4)

    points = pcd[:, :-1]

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    o3d.visualization.draw_geometries([point_cloud])

[38 73]


# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

#### Helper Functions

In [4]:
def get_Rx(theta):
    r_theta = np.matrix(
        [
            [1.0, 0.0, 0.0],
            [0.0, np.cos(theta), -np.sin(theta)],
            [0.0, np.sin(theta), np.cos(theta)],
        ],
        dtype=np.float64,
    )
    r_theta = np.around(r_theta, decimals=4)
    return r_theta


def get_Ry(theta):
    r_theta = np.matrix(
        [
            [np.cos(theta), 0.0, np.sin(theta)],
            [0.0, 1.0, 0.0],
            [-np.sin(theta), 0.0, np.cos(theta)],
        ],
        dtype=np.float64,
    )
    r_theta = np.around(r_theta, decimals=4)
    return r_theta


def get_Rz(theta):
    r_theta = np.matrix(
        [
            [np.cos(theta), -np.sin(theta), 0.0],
            [np.sin(theta), np.cos(theta), 0.0],
            [0.0, 0.0, 1.0],
        ],
        dtype=np.float64,
    )
    r_theta = np.around(r_theta, decimals=3)
    return r_theta


def get_T(R, t=np.array([[0.0], [0.0], [0.0]])):

    return np.vstack(
        (np.concatenate((R, t.reshape(3, 1)), axis=1), np.array([0.0, 0.0, 0.0, 1.0]))
    )


def homogenize(Odometery):
    return np.vstack((Odometery, np.array([0.0, 0.0, 0.0, 1.0])))


def inverse_T(T):
    R = T[:-1, :-1]
    t = T[:-1, -1]

    return np.vstack(
        (np.hstack((R.T, -R.T @ t.reshape(3, 1))), np.array([0.0, 0.0, 0.0, 1.0]))
    )


def inverse(R, t):
    return get_T(R) @ get_T(np.eye(3), -t)

### Transformation matrix to transform LiDar (world) frame to camera frame

In [5]:
R_cw = get_Rx(np.pi / 2) @ get_Rz(np.pi / 2)
T_cw = get_T(R_cw)

print(T_cw)

[[ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]


### Register all point clouds into a common reference frame and visualise it

In [6]:
pcd_list = o3d.geometry.PointCloud()

for index in range(total_frames):
    pcd = np.fromfile(bin_files[index], dtype=np.float32)
    pcd = pcd.reshape(-1, 4)

    ## selecting every 100th point
    points = pcd[:, :-1][::100]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    pcd.transform(homogenize(odometry[index]) @ T_cw)

    pcd_list += pcd

o3d.visualization.draw_geometries([pcd_list])

In [7]:
vis = o3d.visualization.Visualizer()
vis.create_window()

T = get_T(get_Rx(-np.pi / 3))

for index in range(total_frames):
    pcd = np.fromfile(bin_files[index], dtype=np.float32)
    pcd = pcd.reshape(-1, 4)

    # register all point clouds into a common reference frame and visualise it
    points = pcd[:, :-1][::100]

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    point_cloud.transform(T @ homogenize(odometry[index]) @ T_cw)

    vis.add_geometry(point_cloud)

    vis.poll_events()
    vis.update_renderer()

    time.sleep(0.1)
    # vis.capture_screen_image(f'./results/question3/animation/frame_{index}.jpg')

vis.destroy_window()

### Transform the registered point cloud from the world to the $i^{th}$ camera frame

In [8]:
def transform_to_ith_camera(index, pcd):
    R_wc = get_Rz(-np.pi)
    T_wc = get_T(R_wc)

    T_i = T_wc @ inverse_T(homogenize(odometry[index]))
    pcd.transform(T_i)
    
    ## World origin
    mesh_frame1 = o3d.geometry.TriangleMesh().create_coordinate_frame(
        size=20, origin=np.array([0.0, 0.0, 0.0])
    )
    mesh_frame1.transform(T_i)
    
    ## Origin in i-th camera's frame
    mesh_frame2 = o3d.geometry.TriangleMesh().create_coordinate_frame(
        size=10, origin=np.array([0.0, 0.0, 0.0])
    )
    mesh_frame2.transform(T_wc)

    o3d.visualization.draw_geometries([pcd, mesh_frame1, mesh_frame2])

In [9]:
global_pcd = o3d.geometry.PointCloud(pcd_list)
transform_to_ith_camera(75, global_pcd)

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [10]:
threshold = 0.5

In [11]:
global_pcd = o3d.geometry.PointCloud(pcd_list)

In [12]:
def convert_pcd_to_occupancy(pcd):
    pcd_points = np.array(pcd.points)

    x_min, x_max = pcd_points[:, 0].min(), pcd_points[:, 0].max()
    z_min, z_max = pcd_points[:, 2].min(), pcd_points[:, 2].max()

    x_min, x_max = np.floor(x_min).astype(int), np.ceil(x_max).astype(int)
    z_min, z_max = np.floor(z_min).astype(int), np.ceil(z_max).astype(int)

    occupancy_grid = np.zeros((x_max - x_min + 1, z_max - z_min + 1), dtype=np.float64)

    for point in pcd_points:
        x, _, z = point.astype(int)
        occupancy_grid[x - x_min, z - z_min] += 1

    occupancy_grid = np.uint8(occupancy_grid * 255) / 255
    
    return occupancy_grid

### Occupancy map for each LiDAR scan

In [13]:
for index in range(total_frames):
    pcd = np.fromfile(bin_files[index], dtype=np.float32)
    pcd = pcd.reshape(-1, 4)

    points = pcd[:, :-1]

    reflectance = pcd[:, -1]
    colors = np.stack([reflectance, reflectance, reflectance], axis=1)

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    point_cloud.transform(T_cw)

    
    occupancy_grid = convert_pcd_to_occupancy(point_cloud)
    
    fig = plt.figure(figsize=(10, 10))  
    plt.imshow(occupancy_grid > threshold, cmap='gray')
    plt.axis('off')
    plt.title(f'frame_{index} Free')
    plt.savefig(f"./results/occupancy/per_frame_pcd/frame_{index}_free.jpg")
    
    plt.close('all');
    
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(occupancy_grid < threshold, cmap='gray')
    plt.axis('off')
    plt.title(f'frame_{index} Occupied')
    plt.savefig(f"./results/occupancy/per_frame_pcd/frame_{index}_occupied.jpg")
    
    plt.close('all');

### Occupancy map for registered point cloud

In [14]:
occupancy_grid = convert_pcd_to_occupancy(global_pcd)

fig = plt.figure(figsize=(10, 10))  
plt.imshow(occupancy_grid > threshold, cmap='gray')
plt.axis('off')
plt.title(f'Full PCD Free')
plt.savefig(f"./results/occupancy/registered_pcd/full_pcd_free.jpg")

plt.close('all');

fig = plt.figure(figsize=(10, 10))
plt.imshow(occupancy_grid < threshold, cmap='gray')
plt.axis('off')
plt.title(f'Full PCD Occupied')
plt.savefig(f"./results/occupancy/registered_pcd/full_pcd_occupied.jpg")

plt.close('all');

Occupancy map for registered point cloud is more refined compared to the occupancy map for individual point cloud for each frame as the registered point cloud will have detailed information about the environment. 